### DR5 magnitude and shape distributions using extreme deconvolution

In [1]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [3]:
from astropy.io import fits
from astropy.table import Table, join
from sklearn.mixture import GaussianMixture
from desitarget.targetmask import desi_mask, bgs_mask

from astroML.density_estimation import XDGMM

import time
import pickle
import corner as cn

In [143]:
class XD(object):
    
    def __init__(self):
        
        path_to_cuts = 'data/cuts/'
        
        self.lrg = Table.read(path_to_cuts+'lrg_dr5_subset_cuts.fits')
        self.elg = Table.read(path_to_cuts+'elg_dr5_subset_cuts.fits')
        self.bgs = Table.read(path_to_cuts+'bgs_dr5_subset_cuts.fits')
        self.qso = Table.read(path_to_cuts+'qso_dr5_subset_cuts.fits')
        
        self.tags = ['FLUX_G', 'FLUX_R', 'FLUX_Z', 'FLUX_W1', 'FLUX_W2']
        self.err_tags = ['FLUX_IVAR_G', 'FLUX_IVAR_R', 'FLUX_IVAR_Z', 'FLUX_IVAR_W1', 'FLUX_IVAR_W2']
        
        self.shape_tags = ['FRACDEV', 'SHAPEDEV_R', 'SHAPEDEV_E1', 'SHAPEDEV_E2', 'SHAPEEXP_R', 'SHAPEEXP_E1', 
                           'SHAPEEXP_E2']
        self.shape_err_tags = ['FRACDEV_IVAR', 'SHAPEDEV_R_IVAR', 'SHAPEDEV_E1_IVAR', 'SHAPEDEV_E2_IVAR', 
                               'SHAPEEXP_R_IVAR', 'SHAPEEXP_E1_IVAR', 'SHAPEEXP_E2_IVAR']
        
       
    def keep(self, target_type):
        
        
        if target_type == 'QSO':
            data = self.qso
            
            tags = self.tags
            err_tags = self.err_tags
            
            X = np.empty(len(data), dtype=np.dtype([(tt, 'f4') for tt in tags]))
            err = np.empty (len(data), dtype=np.dtype([(et, 'f4') for et in err_tags]))
            
        else:

            if target_type == 'LRG':
                data = self.lrg
            elif target_type == 'ELG':
                data = self.elg
            elif target_type == 'BGS':
                data = self.bgs
                
            tags = self.tags + self.shape_tags
            err_tags = self.err_tags + self.shape_err_tags
                
            X = np.empty(len(data), dtype=np.dtype([(tt, 'f4') for tt in tags]))
            err = np.empty (len(data), dtype=np.dtype([(et, 'f4') for et in err_tags]))
            

        for tt, et in zip(tags, err_tags):
            X[tt] = data[tt]
            err[et] = data[et]
            
        # can clean this up more, possibly split into sub-methods...
       
        keep = np.ones((len(data),), dtype=bool)
        for ii, tt in enumerate(self.tags):
            keep &= (X[tt] > 0)

        X = X[keep]
        err = err[keep]
        
        for jj, tt in enumerate(self.tags):
            X[tt] = 22.5-2.5*np.log10(X[tt])

    
        return(X, err)
    
    def split_by_fracdev(self, data, err, eps):
                
        # condition for devaucouleurs
        dev = data['FRACDEV'] > eps
        dev_tags = self.shape_tags[1:4]
        dev_err_tags = self.shape_err_tags[1:4]
        
        for ii, st in enumerate(dev_tags):
            dev &= (data[st] != 0)
            
        # condition for exponential
        exp = data['FRACDEV'] < eps
        exp_tags = self.shape_tags[4:7]
        exp_err_tags = self.shape_err_tags[4:7]
        
        for ii, st in enumerate(exp_tags):
            exp &= (data[st] != 0)
            
        # condition for composite
        comp = (data['FRACDEV'] >= eps) & (data['FRACDEV'] <= (1-eps))
        comp_tags = self.shape_tags[1:]
        comp_err_tags = self.shape_err_tags[1:]
        
        for ii, st in enumerate(comp_tags):
            comp &= (data[st] != 0)
            
        dev_data = data[dev][self.tags + dev_tags]
        dev_err = err[dev][self.err_tags + dev_err_tags]
        exp_data = data[exp][self.tags + exp_tags]
        exp_err = err[exp][self.err_tags + exp_err_tags]
        comp_data = data[comp][self.tags + comp_tags]
        comp_err = err[comp][self.err_tags + comp_err_tags]

        return(dev_data, dev_err, exp_data, exp_err, comp_data, comp_err)

In [144]:
xd_obj = XD()

In [145]:
X_lrg, X_err = xd_obj.keep('LRG')

In [146]:
lrgDev, lrgDev_err, lrgExp, lrgExp_err, lrgComp, lrgComp_err = xd_obj.split_by_fracdev(X_lrg, X_err, 0.1)